In [1]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

# 1. GENERACIÓN DE DATOS SINTÉTICOS (HISTÓRICO)
# Simulamos una máquina: 
# - Si Temp > 90 y Vibración > 0.8 -> Alta probabilidad de Fallo (1)
np.random.seed(42)
n_samples = 1000

data = pd.DataFrame({
    'temperatura': np.random.uniform(50, 110, n_samples),
    'vibracion': np.random.uniform(0.1, 1.5, n_samples)
})

# Lógica de "Fallo": Un poco de caos + reglas físicas
def generar_etiqueta(row):
    prob_fallo = 0
    if row['temperatura'] > 90: prob_fallo += 0.4
    if row['vibracion'] > 0.8: prob_fallo += 0.4
    # Añadimos ruido aleatorio
    return 1 if (prob_fallo + np.random.uniform(0, 0.3)) > 0.6 else 0

data['fallo'] = data.apply(generar_etiqueta, axis=1)

print(f"📊 Datos generados: {data.shape}")
print(f"Distribución de fallos:\n{data['fallo'].value_counts()}")

# 2. ENTRENAMIENTO DEL MODELO
X = data[['temperatura', 'vibracion']]
y = data['fallo']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Nombre del experimento en MLflow
mlflow.set_experiment("Mantenimiento_Predictivo_v1")

print("\n🚀 Iniciando entrenamiento y registro en MLflow...")

with mlflow.start_run() as run:
    # Entrenar
    clf = RandomForestClassifier(n_estimators=50)
    clf.fit(X_train, y_train)
    
    # Evaluar
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    
    # LOGUEAR A MLFLOW
    mlflow.log_param("n_estimators", 50)
    mlflow.log_metric("accuracy", acc)
    
    # ¡LA MAGIA! Guardamos el modelo entero para usarlo luego
    mlflow.sklearn.log_model(clf, "modelo_prensa")
    
    print(f"✅ Modelo entrenado con Accuracy: {acc:.2f}")
    print(f"📦 Modelo guardado en MLflow Run ID: {run.info.run_id}")
    print(f"🔗 URI del modelo: runs:/{run.info.run_id}/modelo_prensa")

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_config.py:383: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


📊 Datos generados: (1000, 3)
Distribución de fallos:
fallo
0    666
1    334
Name: count, dtype: int64


2026/01/22 11:37:36 INFO mlflow.tracking.fluent: Experiment with name 'Mantenimiento_Predictivo_v1' does not exist. Creating a new experiment.



🚀 Iniciando entrenamiento y registro en MLflow...
✅ Modelo entrenado con Accuracy: 0.87
📦 Modelo guardado en MLflow Run ID: f6d9982a47eb48deb02abb88b4911c3e
🔗 URI del modelo: runs:/f6d9982a47eb48deb02abb88b4911c3e/modelo_prensa
